In [1]:
# import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
import datetime
import pickle
import uuid
import datetime
import numpy as np
import time
import pandas as pd
import tqdm

In [3]:
from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)
import pymilvus

from sentence_transformers import SentenceTransformer


/opt/conda/envs/stackoverflow/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
# from pandarallel import pandarallel

# pandarallel.initialize(progress_bar=True)

In [5]:
path_posts = "gs://np-training-tmp/stackoverflow/final_subset/posts.parquet"
path_posts_related = "gs://np-training-tmp/stackoverflow/final_subset/related_posts.parquet"

In [6]:
collection_name = "stackoverflow"

In [7]:
# def get_model():
#     model = SentenceTransformer('flax-sentence-embeddings/stackoverflow_mpnet-base')
#     return model
    
# def get_milvus_connection(collection_name):
#     milvus_client = Collection(collection_name)
#     return milvus_client

In [8]:
model = SentenceTransformer('flax-sentence-embeddings/stackoverflow_mpnet-base')


In [9]:
text = "Replace me by any question / answer you'd like."
text_embbedding = model.encode(text)

In [10]:
text_embbedding.shape


(768,)

model.

In [11]:
?model.encode

Signature:
model.encode(
    sentences: Union[str, List[str]],
    batch_size: int = 32,
    show_progress_bar: bool = None,
    output_value: str = 'sentence_embedding',
    convert_to_numpy: bool = True,
    convert_to_tensor: bool = False,
    device: str = None,
    normalize_embeddings: bool = False,
) -> Union[List[torch.Tensor], numpy.ndarray, torch.Tensor]
Docstring:
Computes sentence embeddings

:param sentences: the sentences to embed
:param batch_size: the batch size used for the computation
:param show_progress_bar: Output a progress bar when encode sentences
:param output_value:  Default sentence_embedding, to get sentence embeddings. Can be set to token_embeddings to get wordpiece token embeddings. Set to None, to get all output values
:param convert_to_numpy: If true, the output is a list of numpy vectors. Else, it is a list of pytorch tensors.
:param convert_to_tensor: If true, you get one large tensor as return. Overwrites any setting from convert_to_numpy
:param devic

In [12]:
dim = model.get_sentence_embedding_dimension()


In [13]:
df = pd.read_parquet(path_posts)
df['Tags']  = df['Tags'].apply(lambda x: " ".join( x.tolist()))

In [14]:
len(df)

219841

In [15]:
connections.connect("default", host="localhost", port="19530")


In [16]:
utility.list_collections()

['stackoverflow', 'hello_milvus']

In [17]:
if collection_name in utility.list_collections():
    utility.drop_collection(collection_name)

In [18]:
?Collection

Init signature: Collection(name, schema=None, using='default', shards_num=2, **kwargs)
Docstring:      This is a class corresponding to collection in milvus. 
Init docstring:
Constructs a collection by name, schema and other parameters.
Connection information is contained in kwargs.

:param name: the name of collection
:type name: str

:param schema: the schema of collection
:type schema: class `schema.CollectionSchema`

:param using: Milvus link of create collection
:type using: str

:param shards_num: How wide to scale collection. Corresponds to how many active datanodes
                can be used on insert.
:type shards_num: int

:param kwargs:
    * *consistency_level* (``str/int``) --
    Which consistency level to use when searching in the collection. For details, see
    https://github.com/milvus-io/milvus/blob/master/docs/developer_guides/how-guarantee-ts-works.md.
    Options of consistency level: Strong, Bounded, Eventually, Session, Customized.
    Note: this parameter can 

In [19]:
fields = [
    FieldSchema(name="Id", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="AcceptedAnswerId", dtype=DataType.INT64),
    FieldSchema(name="Title", dtype=DataType.VARCHAR, max_length=500),
    FieldSchema(name="QuestionBody", dtype=DataType.VARCHAR, max_length=50_000),
    FieldSchema(name="Tags", dtype=DataType.VARCHAR, max_length=5000),
    FieldSchema(name="ViewCount", dtype=DataType.INT64),
    FieldSchema(name="AnswerCount", dtype=DataType.INT64),
    FieldSchema(name="CommentCount", dtype=DataType.INT64),
    FieldSchema(name="Score", dtype=DataType.INT64),
    FieldSchema(name="AnswerId", dtype=DataType.INT64),
    FieldSchema(name="AcceptedAnswerBody", dtype=DataType.VARCHAR, max_length=50_000),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=dim) ,
    
    #FieldSchema(name="CreationDate", dtype=DataType.VARCHAR),

]

schema = CollectionSchema(fields, "collection containing stackoverflow")

stackoverflow_milvus = Collection(collection_name, schema, consistency_level="Strong")

In [20]:
schema

{
  auto_id: False
  description: collection containing stackoverflow
  fields: [{
    name: Id
    description: 
    type: 5
    is_primary: True
    auto_id: False
  }, {
    name: AcceptedAnswerId
    description: 
    type: 5
  }, {
    name: Title
    description: 
    type: 21
    params: {'max_length': 500}
  }, {
    name: QuestionBody
    description: 
    type: 21
    params: {'max_length': 50000}
  }, {
    name: Tags
    description: 
    type: 21
    params: {'max_length': 5000}
  }, {
    name: ViewCount
    description: 
    type: 5
  }, {
    name: AnswerCount
    description: 
    type: 5
  }, {
    name: CommentCount
    description: 
    type: 5
  }, {
    name: Score
    description: 
    type: 5
  }, {
    name: AnswerId
    description: 
    type: 5
  }, {
    name: AcceptedAnswerBody
    description: 
    type: 21
    params: {'max_length': 50000}
  }, {
    name: embedding
    description: 
    type: 101
    params: {'dim': 768}
  }]
}

In [21]:
fields = [f.name for f in schema.fields]

In [22]:
fields

['Id',
 'AcceptedAnswerId',
 'Title',
 'QuestionBody',
 'Tags',
 'ViewCount',
 'AnswerCount',
 'CommentCount',
 'Score',
 'AnswerId',
 'AcceptedAnswerBody',
 'embedding']

In [23]:

df[['AcceptedAnswerId','AnswerId']] = df[['AcceptedAnswerId','AnswerId']].fillna(-1).astype(int)

cols = ['ViewCount','AnswerCount','CommentCount' ,'Score' ]
df[cols] = df[cols ].fillna(0).astype(int)


df[['AcceptedAnswerBody']] = df[['AcceptedAnswerBody']].fillna("")



In [24]:
df.head()

,Id,AcceptedAnswerId,Title,QuestionBody,Tags,ViewCount,AnswerCount,CommentCount,Score,CreationDate,AnswerId,AcceptedAnswerBody
1,15020895,-1,Python int-byte efficient data structure,i am currently storing key-values of type int-...,python data-structures,155,0,3,1,2013-02-22 09:33:26.360,-1,
9,68487902,-1,Why does the Variance of Laplace very differen...,TL;DR: How can I use skimage.filters.laplace(i...,python opencv image-processing computer-vision...,391,0,5,1,2021-07-22 15:50:34.220,-1,
15,61391327,-1,Why input never ends,I have python 3.7 installed and I have this co...,python python-3.x input,104,1,6,3,2020-04-23 15:43:03.497,-1,
27,28852710,-1,Crashes with piecewise linear objective for gu...,We have a complex optimization problem which i...,python crash gurobi piecewise,403,1,1,3,2015-03-04 10:58:16.370,-1,
29,24043029,-1,Python TypeError: plotdatehist() got an unexpe...,apologies beforehand if this is a stupid quest...,python typeerror,419,0,7,0,2014-06-04 16:42:32.257,-1,


In [25]:
len(df)

219841

In [26]:
df_subset = df.head(5_000_000).copy()

In [27]:
df_subset['Title'].tolist();

In [28]:
# embeddings = np.random.random((len(df_subset),dim))

#embeddings = model.encode(df_subset['Title'], show_progress_bar=True)

embeddings = model.encode(df_subset['Title'].tolist() , show_progress_bar=True)

Batches:   0%|          | 0/6871 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
df_subset['embedding'] = embeddings.tolist()


In [ ]:
df_subset.to_parquet(path_posts.replace(".parquet", "_with_embedding.parquet") , index=False)


In [29]:
df_subset = pd.read_parquet( path_posts.replace(".parquet", "_with_embedding.parquet") )

In [30]:
df_subset.head()

,Id,AcceptedAnswerId,Title,QuestionBody,Tags,ViewCount,AnswerCount,CommentCount,Score,CreationDate,AnswerId,AcceptedAnswerBody,embedding
0,15020895,-1,Python int-byte efficient data structure,i am currently storing key-values of type int-...,python data-structures,155,0,3,1,2013-02-22 09:33:26.360,-1,,"[-0.012659057974815369, -0.006133444607257843,..."
1,68487902,-1,Why does the Variance of Laplace very differen...,TL;DR: How can I use skimage.filters.laplace(i...,python opencv image-processing computer-vision...,391,0,5,1,2021-07-22 15:50:34.220,-1,,"[0.06565586477518082, -0.03163431957364082, 0...."
2,61391327,-1,Why input never ends,I have python 3.7 installed and I have this co...,python python-3.x input,104,1,6,3,2020-04-23 15:43:03.497,-1,,"[0.017581814900040627, -0.07531370967626572, 0..."
3,28852710,-1,Crashes with piecewise linear objective for gu...,We have a complex optimization problem which i...,python crash gurobi piecewise,403,1,1,3,2015-03-04 10:58:16.370,-1,,"[0.010070842690765858, 0.06428135931491852, -0..."
4,24043029,-1,Python TypeError: plotdatehist() got an unexpe...,apologies beforehand if this is a stupid quest...,python typeerror,419,0,7,0,2014-06-04 16:42:32.257,-1,,"[0.03462224081158638, 0.0662810206413269, 0.03..."


In [31]:
df_subset = df_subset [fields]

In [32]:
df_subset.iloc[0].to_dict();

In [33]:
df_subset.dtypes

Id                     int64
AcceptedAnswerId       int64
Title                 object
QuestionBody          object
Tags                  object
ViewCount              int64
AnswerCount            int64
CommentCount           int64
Score                  int64
AnswerId               int64
AcceptedAnswerBody    object
embedding             object
dtype: object

In [34]:
insert_result = stackoverflow_milvus.insert( df_subset  )



In [35]:
insert_result

(insert count: 219841, delete count: 0, upsert count: 0, timestamp: 437015477979185153, success count: 219841, err count: 0)

In [36]:
stackoverflow_milvus.num_entities

219841

In [37]:
stackoverflow_milvus.indexes

[]

https://milvus.io/docs/index.md

In [38]:
# index = {
#     "index_type": "IVF_FLAT",
#     "metric_type": "L2",
#     "params": {"nlist": 128},
# }

index = {
    "index_type": "FLAT",
    "metric_type": "L2",
    "params": {}
}



In [39]:
stackoverflow_milvus.create_index("embedding", index)

Status(code=0, message='Warning: It is not necessary to build index with index_type: FLAT')

In [40]:
?stackoverflow_milvus.create_index

Signature:
stackoverflow_milvus.create_index(
    field_name,
    index_params={},
    timeout=None,
    **kwargs,
) -> pymilvus.orm.index.Index
Docstring:
Creates index for a specified field. Return Index Object.

:param field_name: The name of the field to create an index for.
:type  field_name: str

:param index_params: The indexing parameters.
:type  index_params: dict

:param timeout: An optional duration of time in seconds to allow for the RPC. When timeout
                is set to None, client waits until server response or error occur
:type  timeout: float

:param kwargs:
    * *_async* (``bool``) --
      Indicate if invoke asynchronously. When value is true, method returns a IndexFuture object;
      otherwise, method returns results from server.
    * *_callback* (``function``) --
      The callback function which is invoked after server response successfully. It only take
      effect when _async is set to True.
    * *index_name* (``str``) --
      The name of index which

In [41]:
stackoverflow_milvus.indexes

[]

In [42]:
stackoverflow_milvus.load()


In [43]:
!ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
00_data_fetch_bq.ipynb	   01_data_subset.ipynb     faiss_document_store.db
00_data_fetch_spark.ipynb  02_indexing_es.ipynb     old
01_b_setup.ipynb	   02_indexing_faiss.ipynb  test_setup.ipynb
01_data_cleanup.ipynb	   Untitled.ipynb	    untitled.txt


In [44]:
vectors_to_search = list(df_subset.iloc[0:1]['embedding'])

# search_params = {
#     "metric_type": "L2",
#     "params": {"nprobe": 10},
# }


search_params = {
    "metric_type": "L2",
    #"params": {"nprobe": 128}
    
}



In [45]:
len(vectors_to_search) , len(vectors_to_search[0])

(1, 768)

In [46]:
?stackoverflow_milvus.search;

Object `stackoverflow_milvus.search;` not found.


In [47]:
?time.time

Docstring:
time() -> floating point number

Return the current time in seconds since the Epoch.
Fractions of a second may be present if the system clock provides them.
Type:      builtin_function_or_method


In [48]:
start_time = time.time()
result = stackoverflow_milvus.search(data=vectors_to_search, anns_field="embedding", param=search_params, limit=3
                                     , output_fields=["Id"]
                                    
                                    )
end_time = time.time()

# for hits in result:
#     for hit in hits:
#         print(f"hit: {hit}, random field: {hit.entity.get('random')}")
print((end_time - start_time))

0.3750486373901367


In [49]:
for hits in result:
    for hit in hits:
        print(f"hit: {hit}, score:{hit.score} id: {hit.entity.get('Id')} , data:{hit.entity._row_data} ")

hit: (distance: 0.0, id: 15020895), score:0.0 id: 15020895 , data:{'Id': 15020895} 
hit: (distance: 0.5060367584228516, id: 25471026), score:0.5060367584228516 id: 25471026 , data:{'Id': 25471026} 
hit: (distance: 0.507915198802948, id: 46540337), score:0.507915198802948 id: 46540337 , data:{'Id': 46540337} 


In [50]:
hit.score

0.507915198802948

In [51]:
def format_resp(hits, row):
    payload = []
    query = row['PostTitle']
    
    for hit in hits:
        doc_id = int(hit.entity.get('Id'))
        
        r = {
             'query': query
             , 'query_id' : row['PostId']
             ,'doc_id' : doc_id
             , 'is_relevant' : doc_id in row['RelatedPostIds']
             ,'score' : hit.score
             ,'doc_title' : hit.entity.get('Title')


        }
        payload.append(r)    
    return payload

# def fetch_as_relevancy_eval(row,search_params, num_hits=10):
    
    
#     vectors_to_search = [model.encode( row['PostTitle'])]

    
#     result = stackoverflow_milvus.search(data=vectors_to_search, anns_field="embedding", param=search_params, limit=num_hits
#                                      , output_fields=["Id","Title"]
                                    
#                                     )
 
    
#     payload = format_resp(result, row)
    
#     return pd.DataFrame(payload)
    

# def evaluate_relevancy_hits(df,search_params,num_hits=10):
    
#     payload = []
#     for index, row in df.iterrows():

#         payload_query = fetch_as_relevancy_eval(row,search_params)
        
#         payload.extend(payload_query.to_dict(orient='records') )

    
#     #return pd.DataFrame.from_records(payload)
#     return pd.DataFrame(payload)


# def evaluate_relevancy_hits2_old(df,search_params,num_hits=20):
    
    
#     res = df.parallel_apply(fetch_as_relevancy_eval,num_hits=num_hits,search_params=search_params, axis = 1)

#     return res
    
    
def evaluate_relevancy_hits(df,search_params,num_hits=20, batch_size=10):
    
    payload_all = []
    print(f"Encoding {len(df)} vectors")
    
    for pos in tqdm.trange(0, len(df), batch_size):
        
        df_subset = df.iloc[pos:pos + batch_size] 
    
        vectors_to_search = model.encode( list( df_subset['PostTitle']) )

        result = stackoverflow_milvus.search(data=vectors_to_search, anns_field="embedding", param=search_params, limit=num_hits
                                         , output_fields=["Id","Title"]
                                        )

        for hits , row in zip( result, df_subset.to_dict(orient='records') ):
            payload = format_resp(hits, row)
            payload_all.extend(payload)


    print(f"formatted response")

    df_res = pd.DataFrame(payload_all)
    return df_res

In [52]:
pdf_related = pd.read_parquet(path_posts_related)

In [53]:
pdf_related

,PostId,PostTitle,RelatedPostIds,RelatedPostTitles,num_candidates
1,3494593,Shading a kernel density plot between two points.,"[3494593, 14863744, 14094644, 16504452, 488531...",[Shading a kernel density plot between two poi...,16
2,37949409,Dictionary in a numpy array?,"[37949409, 47689224, 61517741]","[Dictionary in a numpy array?, How to access t...",3
8,19876079,Cannot find module cv2 when using OpenCV,"[19876079, 62443365, 64580641, 45606137, 60294...","[Cannot find module cv2 when using OpenCV, How...",7
12,35082143,Error: package or namespace load failed for ‘car’,"[35082143, 65941744, 68515009, 56409535]",[Error: package or namespace load failed for ‘...,4
14,2673651,inheritance from str or int,"[2673651, 48465797, 3120562, 15085917, 3238350...","[inheritance from str or int, Inherited class ...",15
...,...,...,...,...,...
33231,28419763,Expand Text widget to fill the entire parent F...,"[28419763, 48171462]",[Expand Text widget to fill the entire parent ...,2
33234,40332743,Source code for str.split?,"[40332743, 51355719]","[Source code for str.split?, where can I find ...",2
33241,27443414,Cannot perform a backup or restore operation w...,"[27443414, 53216877]",[Cannot perform a backup or restore operation ...,2
33243,48536681,What is the exact meaning of stride's list in ...,"[48536681, 47305022]",[What is the exact meaning of stride's list in...,2


In [54]:
search_params = {
    "metric_type": "L2",
    #"params": {"nprobe": 128}
    
}

In [55]:
vectors_to_search = model.encode( list( pdf_related.iloc[0:5]['PostTitle']) )

    

In [56]:
result = stackoverflow_milvus.search(data=vectors_to_search, anns_field="embedding", param=search_params, limit=20
                                     , output_fields=["Id","Title"]
                                    )

In [57]:
result

In [58]:
payload_all = []

for hits , row in zip( result, pdf_related.iloc[0:5].to_dict(orient='records') ):
    payload = format_resp(hits, row)
    payload_all.extend(payload)

df_res = pd.DataFrame(payload_all)


In [59]:
df_res

,query,query_id,doc_id,is_relevant,score,doc_title
0,Shading a kernel density plot between two points.,3494593,3494593,True,1.281120e-12,Shading a kernel density plot between two points.
1,Shading a kernel density plot between two points.,3494593,27294822,False,2.057712e-01,Shading a kernel density estimate between two ...
2,Shading a kernel density plot between two points.,3494593,60270301,False,5.561026e-01,Kernel Density Plots and Histogram overlay
3,Shading a kernel density plot between two points.,3494593,7787114,True,6.285322e-01,polygon in density plot?
4,Shading a kernel density plot between two points.,3494593,50526344,False,6.594173e-01,Points with density gradient
...,...,...,...,...,...,...
95,inheritance from str or int,2673651,42359156,False,7.071116e-01,Class inheritance
96,inheritance from str or int,2673651,20604142,False,7.118336e-01,Python Inheritance With No Arguments
97,inheritance from str or int,2673651,65127155,False,7.207956e-01,inheritance with the extention in python
98,inheritance from str or int,2673651,48465797,True,7.299089e-01,Inherited class of int doesn't take additional...


In [60]:
len(pdf_related)


6114

In [61]:
df_res = evaluate_relevancy_hits(pdf_related.iloc[0:50] , search_params=search_params)

Encoding 50 vectors


100%|██████████| 5/5 [00:02<00:00,  1.94it/s]

formatted response


In [63]:
df_res = evaluate_relevancy_hits(pdf_related , search_params=search_params)

Encoding 6114 vectors


100%|██████████| 612/612 [04:41<00:00,  2.17it/s]

formatted response


In [65]:

df_agg_res  = df_res.groupby(['query_id'], as_index=False).apply (lambda x: pd.Series(metrics_utils.all_metrics(x['is_relevant'])))


In [66]:
df_agg_res

,query_id,p@1,p@5,p@10,mrr
0,972,1.0,0.6,0.4,1.0
1,8948,1.0,0.6,0.3,1.0
2,20794,1.0,0.4,0.2,1.0
3,32404,1.0,0.4,0.2,1.0
4,32899,1.0,0.6,0.4,1.0
...,...,...,...,...,...
6109,71792480,1.0,0.2,0.1,1.0
6110,71992622,1.0,0.4,0.2,1.0
6111,72050038,1.0,0.2,0.1,1.0
6112,72369460,1.0,0.2,0.2,1.0


In [67]:
df_agg_res.drop(columns='query_id').agg(np.mean)

p@1     0.999509
p@5     0.294799
p@10    0.169038
mrr     0.999755
dtype: float64

In [68]:
connections.disconnect('default')

In [74]:
connections.list_connections()

[('default', None)]

### Speeding up

In [75]:
connections.connect("default", host="localhost", port="19530")


In [76]:
index_ivf_flat = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}


In [77]:
stackoverflow_milvus.create_index("embedding", index_ivf_flat)

Status(code=0, message='')

In [78]:
resp=stackoverflow_milvus.indexes[0]

In [79]:
resp.to_dict()

{'collection': 'stackoverflow',
 'field': 'embedding',
 'index_name': '_default_idx',
 'index_param': {'index_type': 'IVF_FLAT',
  'metric_type': 'L2',
  'params': {'nlist': 128}}}

In [80]:
stackoverflow_milvus.load()


In [81]:
search_params_ivf_flat = {
    "metric_type": "L2",
    "params": {"nprobe": 1}
    
}

In [82]:
df_res = evaluate_relevancy_hits(pdf_related , search_params=search_params_ivf_flat)

Encoding 6114 vectors


100%|██████████| 612/612 [04:39<00:00,  2.19it/s]

formatted response


In [83]:
df_agg_res.drop(columns='query_id').agg(np.mean)

p@1     0.999509
p@5     0.294799
p@10    0.169038
mrr     0.999755
dtype: float64

In [84]:
df_agg_res.to_parquet("../tmp/df_agg_res__faiss.parquet", index=False)
df_agg_res.head()

,query_id,p@1,p@5,p@10,mrr
0,972,1.0,0.6,0.4,1.0
1,8948,1.0,0.6,0.3,1.0
2,20794,1.0,0.4,0.2,1.0
3,32404,1.0,0.4,0.2,1.0
4,32899,1.0,0.6,0.4,1.0


In [85]:
df_res.to_parquet("../tmp/df_res__faiss.parquet", index=False)
df_res.head()

,query,query_id,doc_id,is_relevant,score,doc_title
0,Shading a kernel density plot between two points.,3494593,3494593,True,1.383861e-12,Shading a kernel density plot between two points.
1,Shading a kernel density plot between two points.,3494593,27294822,False,2.057712e-01,Shading a kernel density estimate between two ...
2,Shading a kernel density plot between two points.,3494593,60270301,False,5.561024e-01,Kernel Density Plots and Histogram overlay
3,Shading a kernel density plot between two points.,3494593,7787114,True,6.285320e-01,polygon in density plot?
4,Shading a kernel density plot between two points.,3494593,50526344,False,6.594170e-01,Points with density gradient


cant add new fields     
order of fields matter    
field size matters    

**Can vectors with duplicate primary keys be inserted into Milvus?**    
Yes. Milvus does not check if vector primary keys are duplicates.


**When vectors with duplicate primary keys are inserted, does Milvus treat it as an update operation?**
No. Milvus does not currently support update operations and does not check if entity primary keys are duplicates. You are responsible for ensuring entity primary keys are unique, and if they aren't Milvus may contain multiple entities with duplicate primary keys.

If this occurs, which data copy will return when queried remains an unknown behavior. This limitation will be fixed in future releases.

https://milvus.io/docs/product_faq.md#Can-vectors-with-duplicate-primary-keys-be-inserted-into-Milvus

In [ ]:
# reference

https://github.com/milvus-io/pymilvus/blob/master/examples/hello_milvus.ipynb


https://milvus.io/tools/sizing/